# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# from torchvision.models import resnet34

# test = resnet34().cuda()

# import torch

# out = test(torch.ones([3,3,224,224]).cuda())

In [8]:
from fastai.text import *

In [9]:
path = Path('data/composers/notewise/piano_solo/note_range38/sample_freq12')

In [10]:
bs=4
bptt=20

In [11]:
data = TextLMDataBunch.load(path, bs=bs)

In [12]:
vocab = data.train_ds.vocab
vocab_sz = len(vocab.itos); vocab_sz

110

In [13]:
t = data.train_ds[0][0]
t.text[:50], t.data

('xxbos wait25 wait25 wait25 wait25 wait25 wait25 wa',
 array([  2,  94,  94,  94, ...,   9,  53,   9, 109]))

In [8]:
data.train_dl

DeviceDataLoader(dl=<fastai.text.data.LanguageModelLoader object at 0x7f6710c111d0>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7f6717b29b70>)

In [14]:
class LMTrainer(LearnerCallback):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn):
        super().__init__(learn)
        
    
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        "Applies mixup to `last_input` and `last_target` if `train`."
        position_ids = torch.arange(last_input.shape[-1], dtype=torch.long, device=last_input.device)
        position_ids = position_ids.unsqueeze(0).expand_as(last_input)
        target_ids = torch.arange(last_target.shape[-1], dtype=torch.long, device=last_target.device)
        target_ids = target_ids.unsqueeze(0).expand_as(last_target)
        return (last_input, position_ids, last_target, target_ids), last_target[:, 1:]

In [8]:
# data.show_batch(rows=1)

### Sanity check

In [39]:
learn = language_model_learner(data, drop_mult=1, clip=.5, bptt=bptt)

ob = data.one_batch()

out = learn.model(ob[0].cuda())

In [56]:
ob[1].shape

torch.Size([4, 67])

In [42]:
out[0].shape

torch.Size([4, 67, 110])

In [41]:
[x.shape for x in out[1]]

[torch.Size([4, 67, 1150]),
 torch.Size([4, 67, 1150]),
 torch.Size([4, 67, 400])]

In [ ]:
out[1].shape

### Transformer Arch
Paper: https://arxiv.org/abs/1706.03762  
Inspiration: https://github.com/jadore801120/attention-is-all-you-need-pytorch

In [15]:
import transformer.Constants as Constants
# from dataset import TranslationDataset, paired_collate_fn
from transformer.Models import Transformer
from transformer.Optim import ScheduledOptim

In [16]:
transformer = Transformer(
    n_src_vocab=vocab_sz,
    n_tgt_vocab=vocab_sz,
    len_max_seq=bptt+100,
#     len_max_seq=opt.max_token_seq_len,
#     tgt_emb_prj_weight_sharing=opt.proj_share_weight,
#     emb_src_tgt_weight_sharing=opt.embs_share_weight,
#     d_k=opt.d_k,
#     d_v=opt.d_v,
#     d_model=opt.d_model,
#     d_word_vec=opt.d_word_vec,
#     d_inner=opt.d_inner_hid,
#     n_layers=opt.n_layers,
#     n_head=opt.n_head,
#     dropout=opt.dropout
)

In [17]:
# transformer = transformer.cuda()
transformer.reset = lambda: None

In [45]:
from transformer.Models import *

In [46]:
transformer

Transformer(
  (encoder): Encoder(
    (src_word_emb): Embedding(110, 512, padding_idx=0)
    (position_enc): Embedding(21, 512)
    (layer_stack): ModuleList(
      (0): EncoderLayer(
        (slf_attn): MultiHeadAttention(
          (w_qs): Linear(in_features=512, out_features=512, bias=True)
          (w_ks): Linear(in_features=512, out_features=512, bias=True)
          (w_vs): Linear(in_features=512, out_features=512, bias=True)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1)
            (softmax): Softmax()
          )
          (layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
          (fc): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (pos_ffn): PositionwiseFeedForward(
          (w_1): Conv1d(512, 2048, kernel_size=(1,), stride=(1,))
          (w_2): Conv1d(2048, 512, kernel_size=(1,), stride=(1,))
          (layer_norm): LayerNorm(torch.Size([512

In [48]:
src_pos = torch.zeros_like(ob[0])
tgt_pos = torch.zeros_like(ob[1])

In [50]:
out = transformer(ob[0], src_pos, ob[1], tgt_pos)

In [54]:
4*66

264

In [51]:
out.shape

torch.Size([264, 110])

In [52]:
out[1].shape

torch.Size([110])

In [36]:
%debug

> /home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/torch/nn/functional.py(1194)linear()
   1192         ret = torch.addmm(torch.jit._unwrap_optional(bias), input, weight.t())
   1193     else:
-> 1194         output = input.matmul(weight.t())
   1195         if bias is not None:
   1196             output += torch.jit._unwrap_optional(bias)

ipdb> input.shape
torch.Size([4, 67, 512])
ipdb> input.type
<built-in method type of Tensor object at 0x7f205e284048>
ipdb> input.type()
'torch.cuda.FloatTensor'
ipdb> weight.type()
'torch.cuda.FloatTensor'
ipdb> weight.shape
torch.Size([512, 512])
ipdb> weight.t().shape
torch.Size([512, 512])
ipdb> quit


In [26]:
out[0]

tensor([[  2,  10,  27,  31,   9,  53,   8,  46,  29,   9,  26,  51,  31,  10,
          45,   8,  16,  10,  35,   8,  26,  10,  45,   8,  27,  31,  10,  46,
           8,  22,  10,  40,   8,  21,  31,   9,  29,  53,   8,  44,   8,  51,
           8,  22,  31,  10,  40,  27,  10,  46,  10,  22,  10,  21,  40,  31,
          10,  44,  10,  28,   8,  53,   9,  42,  52,  31,   9],
        [ 10,  39,  56,   8,  29,  51,   9,  39,  56,   9,  29,   8,  29,  51,
          39,  56,  10,  29,  51,  39,  56,  10,  39,  56,   8,  29,  51,   9,
          39,  56,   9,  29,   8,  29,  51,  39,  56,   9,  38,   8,  50,  26,
          45,  16,  35,  17,  29,  51,  39,  56,  10,  16,  35,  39,  56,   8,
          26,  45,  29,  51,   9,  16,  35,  39,  56,   9,  26],
        [ 63,  90,  95,  91,  70,   9,  79,  11,  92,   8,  74, 103,  76,   8,
          89,  94,  94,  93,  62,  99,  70,  69,  94, 100,  74,   8,  76,  94,
          87,  90,   8,  79,  94,  99,  18,  29,  60,  83,  37,   8,  51,  65,
 

In [29]:
tgt_seq = out[1]
src_seq = out[0]

In [30]:

# dec_slf_attn_list, dec_enc_attn_list = [], []

# # -- Prepare masks
# non_pad_mask = get_non_pad_mask(tgt_seq)

# slf_attn_mask_subseq = get_subsequent_mask(tgt_seq)
# slf_attn_mask_keypad = get_attn_key_pad_mask(seq_k=tgt_seq, seq_q=tgt_seq)
# slf_attn_mask = (slf_attn_mask_keypad + slf_attn_mask_subseq).gt(0)

# dec_enc_attn_mask = get_attn_key_pad_mask(seq_k=src_seq, seq_q=tgt_seq)

# # slf_attn_mask = triangular thing
# # dec_enc_attn_mask = all zeroes - always use encoder weights

In [31]:
dec_enc_attn_mask

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

In [ ]:
    def forward(self, tgt_seq, tgt_pos, src_seq, enc_output, return_attns=False):

        dec_slf_attn_list, dec_enc_attn_list = [], []

        # -- Prepare masks
        non_pad_mask = get_non_pad_mask(tgt_seq)

        slf_attn_mask_subseq = get_subsequent_mask(tgt_seq)
        slf_attn_mask_keypad = get_attn_key_pad_mask(seq_k=tgt_seq, seq_q=tgt_seq)
        slf_attn_mask = (slf_attn_mask_keypad + slf_attn_mask_subseq).gt(0)

        dec_enc_attn_mask = get_attn_key_pad_mask(seq_k=src_seq, seq_q=tgt_seq)

        # -- Forward
        dec_output = self.tgt_word_emb(tgt_seq) + self.position_enc(tgt_pos)

        for dec_layer in self.layer_stack:
            dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                dec_output, enc_output,
                non_pad_mask=non_pad_mask,
                slf_attn_mask=slf_attn_mask,
                dec_enc_attn_mask=dec_enc_attn_mask)

            if return_attns:
                dec_slf_attn_list += [dec_slf_attn]
                dec_enc_attn_list += [dec_enc_attn]

        if return_attns:
            return dec_output, dec_slf_attn_list, dec_enc_attn_list
        return dec_output,

In [ ]:
tras

In [27]:
# learn = LanguageLearner(data, model, bptt, split_func=lm_split, **kwargs)

### Create Language learner

In [18]:
learn = LanguageLearner(data, transformer, bptt)

In [19]:
learn.callbacks = [LMTrainer(learn)]

In [20]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy


KeyboardInterrupt: 

In [19]:
%debug

> <ipython-input-10-cf5d12bf5ada>(12)on_batch_begin()
      9         position_ids = torch.arange(last_input.shape[-1], dtype=torch.long, device=last_input.device)
     10         position_ids = position_ids.unsqueeze(0).expand_as(last_input)
     11         target_ids = torch.arange(last_target.shape[-1], dtype=torch.long, device=last_target.device)
---> 12         target_ids = last_target.unsqueeze(0).expand_as(last_target)
     13         return (last_input, position_ids, last_target, target_ids), last_target[:, 1:]

ipdb> last_target.shape
torch.Size([4, 95])
ipdb> position_ids.shape
torch.Size([4, 95])
ipdb> last_input.shape
torch.Size([4, 95])
ipdb> target_ids.shape
torch.Size([95])
ipdb> quit
